### Selected Word Group In Youtube Sentence

In [33]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [62]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6
sample_num = 6

# youtube sentence condition
sent_time_start = True  # True, False
sent_time_start_time = 700 # sn

In [35]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [36]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [37]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [38]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [39]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [40]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [41]:
if sent_time_start:
    df_youtube_sentence = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_time_start_time]
    df_youtube_sentence.reset_index(drop=True, inplace=True)
else:
    df_youtube_sentence = df_youtube_sentence

df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
1050454,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
1050455,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
1050456,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
1050457,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [42]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,adam yine geldi
1,al bakalım dostum
2,al şunu al
3,al şunu da
4,ama asla olmaz
...,...
485,yoksa kötü mü
486,yoksa sizi de
487,zaman bugün mü
488,zaman seninle olsun


In [43]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [44]:
len(search_list)

490

In [45]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,al şunu al,571.250,573.240,fatma abla al şunu al,eBKS1dCabuI
1,al şunu da,689.546,690.515,al şunu da hadi ben işe gidiyorum,EtWqWE8gdF0
2,al şunu da,657.980,659.080,haydi al şunu da,OcZJIp3KUYs
3,al şunu da,642.889,644.749,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
4,ama aslında gerçek,75.264,81.376,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
...,...,...,...,...,...
1330,yok tamam mı,351.520,353.000,ama acele etmek yok tamam mı,ZmahvhKmnns
1331,yok tamam mı,274.780,278.560,şirin benim sana yalan borcum yok tamam mı ben...,mXgr8WVo8D4
1332,yoksa kötü mü,205.655,206.760,yoksa kötü mü olur bilemiyorum,jcqVzKEV9kI
1333,yoksa kötü mü,671.106,672.524,yoksa kötü mü,aPwTtMXPv3o


In [46]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,al şunu al,572.197619,573.240000,fatma abla al şunu al,eBKS1dCabuI
1,al şunu da,689.546000,689.869000,al şunu da hadi ben işe gidiyorum,EtWqWE8gdF0
2,al şunu da,658.323750,659.080000,haydi al şunu da,OcZJIp3KUYs
3,al şunu da,643.789000,644.509000,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
4,ama aslında gerçek,75.264000,76.876889,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
...,...,...,...,...,...
1330,yok tamam mı,352.312857,353.000000,ama acele etmek yok tamam mı,ZmahvhKmnns
1331,yok tamam mı,276.346000,277.102000,şirin benim sana yalan borcum yok tamam mı ben...,mXgr8WVo8D4
1332,yoksa kötü mü,205.655000,206.170667,yoksa kötü mü olur bilemiyorum,jcqVzKEV9kI
1333,yoksa kötü mü,671.106000,672.524000,yoksa kötü mü,aPwTtMXPv3o


In [47]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,al şunu al,571.597619,573.840000,fatma abla al şunu al,eBKS1dCabuI
1,al şunu da,688.946000,690.469000,al şunu da hadi ben işe gidiyorum,EtWqWE8gdF0
2,al şunu da,657.723750,659.680000,haydi al şunu da,OcZJIp3KUYs
3,al şunu da,643.189000,645.109000,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
4,ama aslında gerçek,74.664000,77.476889,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
...,...,...,...,...,...
1330,yok tamam mı,351.712857,353.600000,ama acele etmek yok tamam mı,ZmahvhKmnns
1331,yok tamam mı,275.746000,277.702000,şirin benim sana yalan borcum yok tamam mı ben...,mXgr8WVo8D4
1332,yoksa kötü mü,205.055000,206.770667,yoksa kötü mü olur bilemiyorum,jcqVzKEV9kI
1333,yoksa kötü mü,670.506000,673.124000,yoksa kötü mü,aPwTtMXPv3o


In [48]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,al şunu al,572,574,fatma abla al şunu al,eBKS1dCabuI
1,al şunu da,689,690,al şunu da hadi ben işe gidiyorum,EtWqWE8gdF0
2,al şunu da,658,660,haydi al şunu da,OcZJIp3KUYs
3,al şunu da,643,645,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg
4,ama aslında gerçek,75,77,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
...,...,...,...,...,...
1330,yok tamam mı,352,354,ama acele etmek yok tamam mı,ZmahvhKmnns
1331,yok tamam mı,276,278,şirin benim sana yalan borcum yok tamam mı ben...,mXgr8WVo8D4
1332,yoksa kötü mü,205,207,yoksa kötü mü olur bilemiyorum,jcqVzKEV9kI
1333,yoksa kötü mü,671,673,yoksa kötü mü,aPwTtMXPv3o


In [49]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,al şunu al,572,574,fatma abla al şunu al,eBKS1dCabuI,https://www.youtube.com/watch?v=eBKS1dCabuI&t=...
1,al şunu da,689,690,al şunu da hadi ben işe gidiyorum,EtWqWE8gdF0,https://www.youtube.com/watch?v=EtWqWE8gdF0&t=...
2,al şunu da,658,660,haydi al şunu da,OcZJIp3KUYs,https://www.youtube.com/watch?v=OcZJIp3KUYs&t=...
3,al şunu da,643,645,hadi aç bakayım al şunu da ısır,5e5vf1WrQYg,https://www.youtube.com/watch?v=5e5vf1WrQYg&t=...
4,ama aslında gerçek,75,77,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI,https://www.youtube.com/watch?v=yXkiaO7_SKI&t=75s
...,...,...,...,...,...,...
1330,yok tamam mı,352,354,ama acele etmek yok tamam mı,ZmahvhKmnns,https://www.youtube.com/watch?v=ZmahvhKmnns&t=...
1331,yok tamam mı,276,278,şirin benim sana yalan borcum yok tamam mı ben...,mXgr8WVo8D4,https://www.youtube.com/watch?v=mXgr8WVo8D4&t=...
1332,yoksa kötü mü,205,207,yoksa kötü mü olur bilemiyorum,jcqVzKEV9kI,https://www.youtube.com/watch?v=jcqVzKEV9kI&t=...
1333,yoksa kötü mü,671,673,yoksa kötü mü,aPwTtMXPv3o,https://www.youtube.com/watch?v=aPwTtMXPv3o&t=...


In [64]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

317

In [56]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [59]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

,not_in_youtube_result
0,küçük kız vardı
1,haydi git buradan
2,o bize lazım
3,iyi olacak mısın
4,tabii eğer o
...,...
168,biri olabilir diye
169,haydi gidelim buradan
170,onunla birlikte değil
171,yoksa bizi mi


In [61]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

173

In [65]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [66]:
df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
Not_ın_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [67]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_317_Threegram_6_Youtube_0.6s_Timeshift_Result.xlsx',
 'Turkish_173_Threegram_Not_ın_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [68]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [69]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass